# An intro to RNNs

This notebook will serve as an accumulation of all the notes that I have kept over time from various sources about RNNs. Look at the cell below for an accumulation of all those resources.

## Resources

* Personal Notes from Stanford's [CS224](http://web.stanford.edu/class/cs224n/) (Natural Language Processing with Deep Learning)

* A nice [article](http://colah.github.io/posts/2015-08-Understanding-LSTMs/) by Christopher Olah

* A nice [article](http://blog.echen.me/2017/05/30/exploring-lstms/) from Edwin Chen.

# Vanilla RNNs

### Some history

In the days before RNNs, if we wanted to concern ourselves with language modeling, namely predicting the next word in a sentence then we would employ mostly some count based approaches to do so. We would extract certain n-grams from our corpus and then we would simply have some lookup using base frequencies to measure which word is most likely to follow. As an example, if I were working with bigrams and trigrams and I wanted to know what is the next word in the sentence ``` "My name ..."``` then I would go back to my corpus, and calculate 

$$p(w | my, name) = \frac{count(My, name, w)}{count(My, name)}$$

or in plain words, we would find a candidate trigram and we would derive its probability by dividing that count with the count of the bigram we have observed. However, counting n-grams in our corpus in the first place is itself very challenging computationally so we need smarter ways to do that!

### RNNs
RNNs are particularly good when working with time series data, or data in general that have some temporal order. An example could be, I want to measure the progress of a patient with his/her treatment using as my data the reports that the doctor is writing about the patient over time. (imagine we have obtained distributed representation of those reports using using word embeddings). Clearly reports written more recently will be more indicative of the patient'state of recovery at the moment. So we need a network that can exploit that temporal nature of our data to reach the correct conclusion.

The main characteristic of RNNs for me is their hidden state (think of a vector) that gets updated as the data is being fed into the network. This allows the network to encode information from previous states into the current state.


![Diagramatically](imgs/vanilla_rnn.png)

Notice that the above image shows the state of the network at different timesteps when we feed in different inputs. Mathematically, the way you update a hidden state is some 

$$ f(h_{t-1}, W^{(hh)}, W^{(hx)}, x(t)) = \sigma(W^{(hh)}h_{t-1} + W^{(hx)}x(t))$$

Let's unpack this.

Assume the hidden state $h(t) \in \mathbb{R}^{h \times 1}$ and the input at time $t$ $x(t) \in \mathbb{R}^{x \times 1}$.

Now, $W^{(hh)} \in \mathbb{R}^{h \times h}$ and $W^{(hx)} \in \mathbb{R}^{h \times x}$ are learnable parameters of the network.

Now notice that $W^{(hh)}h_{t-1}$ and $W^{(hx)}x(t)$ yield column vectors of dimension $h$ by $1$ which is then taken through an element-wise non-linearity, the sigmoid function.

Hence, we are using both our previous hidden state and the input at time $t$ to update our hidden state $h_{t}$.

In the end of the network, we typically have some softmax layer that gives us the probability of each word in our vocabulary being the next word in the sequence.

So when I say that I mean some operation $$probabilities = softmax(W_{out}h_t + b_{out})$$ 

where $W_{out}$ and $b_{out}$ are learnable parameters and then we treat the outcome of that linear operation as unnormalized log-probabilities and we take them through the softmax. The softmax activation is a classic move in DL/ML that allows us to get a probability distribution in our output layer.

To evaluate the prediction of the RNN we will use a loss function called cross entropy.

$$J = - \frac{1}{T} \sum_{t}\sum_{v} y_{t, v} log(\hat{y}_{t, v})$$

This needs a little bit of clarification.

Given a vocabulary size of $V$ and a $T$ prediction problebms,

$y$ is a $T \times V$ matrix where each row represents the one hot encoded representation of the words we are trying to predict.

$\hat{y}$ on the other hand is the also a $T \times V$ matrix but contains the probability distribution we generated for each word we wanted to predict.

Now what the cross entropy loss does is that for each word we are trying to predict, it calculates the logarithm of the probability of the correct word as the loss and then averages across the loss for all predictions. This makes sense because ideally our network would predict the correct word with probability 1 and so the loss for that prediction would be $log 1 = 0$.

More commonly you will also see perplexity being used which is nothing more than $2^J$.

Training RNNs is hard! The training process is often bottlenecked by the **vanishing gradient problem**. Another issue is that RNNs don't really have a long term memory so it is hard to train on long sequences. As a result we tend to use some better variants like Long Short Term Memory networks (LSTMs) or Gated Recurrent Units (GRUs). That's why we tend to not use traditional 